# HAM AND SPAM

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import tensorflow as tf
import keras.models
from keras.layers import Conv2D , MaxPool2D , Flatten , Dense , Dropout

from keras.layers import SimpleRNN , GRU , LSTM , Embedding
from keras.models import Model , Sequential
from PIL import Image 
from keras.layers import GlobalAveragePooling2D , BatchNormalization
from tensorflow.keras.utils import to_categorical

In [109]:
# Read the File
messages  = pd.read_csv(r"E:\PGA $$ 23 $$\Deep Learning\New folder\spam1.csv",encoding ='cp1252')
messages.shape

(6776, 5)

In [110]:
messages = messages.iloc[: ,[0,1]]
messages.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [111]:
messages.rename(columns={'v1':'Label',"v2":'Message'},inplace = True)
messages.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [112]:
messages.Label.replace({"ham":1,"spam":0},inplace = True)


In [113]:
# SAMPLING INTO TRAIN TEST
messages_x = messages.iloc[: , 1]
messages_y = messages.iloc[: ,0]
from sklearn.model_selection import train_test_split
x_train ,x_test ,y_train ,y_test= train_test_split(messages_x , messages_y , test_size = .2)

In [114]:
# Checking the records to match the records in train and test
print(x_train.shape)
print(y_train.shape)
print("-----------")
print(x_test.shape)
print(y_test.shape)


(5420,)
(5420,)
-----------
(1356,)
(1356,)


In [115]:
## Coverting into categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [116]:
print(y_train.shape)
print(y_test.shape)

(5420, 2)
(1356, 2)


In [117]:
# APPLYING SOME HYPERPARAMETER
max_num_words = 10000  ## how many words do we consider from the entire CORPUS
seq_len = 50 ## how many tokens(words to consider in each documents / message)
embedding_size = 100  ## vector length of each words

from keras.preprocessing.text import Tokenizer # is used to assign id to each unique words(tokenization)
from keras.preprocessing.sequence import pad_sequences # used to bring all samples to same length

In [118]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(messages.Message)
x_train = tokenizer.texts_to_sequences(x_train) # will convert the text to sequences of IDs
x_train = pad_sequences(x_train , maxlen = seq_len)

x_test = tokenizer.texts_to_sequences(x_test) # will convert the text to sequences of IDs
x_test = pad_sequences(x_test , maxlen = seq_len)

model = Sequential()   # initialize the network
model.add(Embedding(input_dim = max_num_words , 
                   input_length =seq_len  , output_dim = embedding_size))

In [119]:
model.add(LSTM(5))
model.add(Dense(2 , activation = 'softmax'))
model.compile(optimizer ='adam' , loss = 'categorical_crossentropy',metrics=['accuracy'])

model.fit(x_train , y_train , epochs=5 ,batch_size = 32 , validation_split = .2)

Epoch 1/5
136/136 [==============================] - 8s 41ms/step - loss: 0.2455 - accuracy: 0.9197 - val_loss: 0.0888 - val_accuracy: 0.9843
Epoch 2/5
136/136 [==============================] - 5s 35ms/step - loss: 0.0522 - accuracy: 0.9910 - val_loss: 0.0554 - val_accuracy: 0.9908
Epoch 3/5
136/136 [==============================] - 5s 40ms/step - loss: 0.0210 - accuracy: 0.9977 - val_loss: 0.0462 - val_accuracy: 0.9917
Epoch 4/5
136/136 [==============================] - 5s 40ms/step - loss: 0.0100 - accuracy: 0.9991 - val_loss: 0.0460 - val_accuracy: 0.9899
Epoch 5/5
136/136 [==============================] - 5s 34ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.0456 - val_accuracy: 0.9899


In [120]:
pred= model.predict(x_test)
from sklearn.metrics import confusion_matrix
tab = confusion_matrix(np.argmax(pred,1),np.argmax(y_test ,axis =1))
tab

array([[ 194,    0],
       [  12, 1150]], dtype=int64)

In [121]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(np.argmax(pred,1),np.argmax(y_test,1))
acc

0.9911504424778761